# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

In [1]:
from sklearn.decomposition  import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_boston, load_iris, load_breast_cancer, make_blobs
import numpy as np
from random import randrange
from sklearn.metrics import accuracy_score
import cvxpy as cp

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = load_iris()
X, y = data.data, data.target

# PCA

In [2]:
class PrincipalComponentAnalysis():
    def __init__(self, n_components):
        self.n_components = n_components
        self.components = None
    
    def fit(self, X):
        Xc = X - np.mean(X, axis=0)
        _, _, self.components = np.linalg.svd(Xc)
    
    def transform(self, X):
        trans = self.components @ X.T
        return trans[:self.n_components, :].T

In [3]:
X_centered = X - np.mean(X, axis=0)

pca = PCA(n_components=3)
pca.fit(X_centered)
X_pca_trans = pca.transform(X_centered)

model = PrincipalComponentAnalysis(n_components=3)
model.fit(X_centered)
X_model_trans = model.transform(X_centered)

sign_correct_X_model_trans = np.concatenate([np.expand_dims(X_model_trans[:,0],axis=1),-X_model_trans[:,1:]],axis=1)

error = rel_error(X_pca_trans, sign_correct_X_model_trans)
print(error)
assert  error < 1e-11

1.5352932325130042e-13


# Binary Linear SVM with CVXPY

## Hard margin 

In [4]:
X2, y2 = make_blobs(n_samples=300, centers=2, n_features=12, random_state=47)
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)
y2[y2 == 0] = -1

$$\min_{\mathbf{w},b}\frac{1}{2}||\mathbf{w}||^2$$ <br>
$$\text{s.t } y_i(\mathbf{w}^{\top}\mathbf{x}_i + b) \ge 1, \ i=1...N$$

In [5]:
class LinearSVMHardMargin():
    def __init__(self):
        self.w = None
        self.b = 0
    
    def fit(self, X, y):
        
        N, d = X.shape
        
        w = cp.Variable(d)
        objective = cp.Minimize(cp.sum_squares(w) / 2)
        constraints = [
            y[i] * (w @ X[i] + self.b) >= 1 for i in range(N)
        ]
        
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        self.w = w.value
        
    def predict(self, X):
        """Return the predicted label 1 or -1"""
        return np.sign(X @ self.w + self.b)

In [6]:
model = LinearSVMHardMargin()
model.fit(X2, y2)
pred = model.predict(X2)
accuracy = accuracy_score(y2, pred)
print(accuracy)
assert accuracy == 1

1.0


## Soft Margin

In [7]:
data3 = load_breast_cancer()
X3, y3 = data3.data, data3.target
scaler = StandardScaler()
X3 = scaler.fit_transform(X3)
y3[y3 == 0] = -1

$$L(\mathbf{w},b) = \frac{1}{N} \sum_{i=1}^N \max(0, 1 - y_i(\mathbf{w}^{\top}\mathbf{x}_i + b)) + \lambda||\mathbf{w}||^2_2$$

In [8]:
class LinearSVMSoftMargin():
    def __init__(self, alpha=0):
        self.w = None
        self.b = 0
        self.alpha = alpha
    
    def fit(self, X, y):
        
        N, d = X.shape
        
        w = cp.Variable(d)
        b = cp.Variable(1)
        
        objective = cp.Minimize(
            cp.mean(cp.maximum(0, 1 - cp.multiply(y, X @ w + b))) + self.alpha * cp.sum_squares(w)
        )
        
        problem = cp.Problem(objective)
        problem.solve()
        
        self.w = w.value
        self.b = b.value
        
    def predict(self, X):
        """Return the predicted label 1 or -1"""
        return np.sign(X @ self.w + self.b)

In [9]:
model = LinearSVMSoftMargin(alpha=1e-3)
model.fit(X3, y3)
pred = model.predict(X3)
accuracy = accuracy_score(y3, pred)
print(accuracy)
assert accuracy >= 0.98

0.9876977152899824


# Multiclass Linear SVM

## Loss

$$L(\mathbf{W}) = \sum_{i=1}^N \sum_{j \neq y_i} \max(0, s_j - s_{y_i} + 1) + \lambda||\mathbf{w}||^2_2$$ <br>
$$\text{where } s_j = (f(\mathbf{x}_i;\mathbf{W}))_j = (\mathbf{W}\mathbf{x}_i)_j \text{ is the score for the j-th class}$$

In [10]:
W = np.random.randn(X.shape[1], 3) * 0.0001

In [22]:
def svm_loss_naive(W, X, y, alpha):
    """
    Multiclass SVM loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    
    
    D, C = W.shape
    N = X.shape[0]
    
    for i in range(N):
        c = 0.0
        for n in range(C):
            M = W[:,n] @ X[i] - W[:,y[i]] @ X[i] + (n != y[i])
            loss += max(0, M)
            c += (M > 0)
            dW[:, n] += (1 - (n == y[i])) * X[i] if M >= 0 else 0
        dW[:, y[i]] -= X[i] * c
            
    loss += np.sum(alpha * W**2)
    dW += 2 * alpha * W
    
    return loss, dW

In [23]:
# NO REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 0.0)

f = lambda W: svm_loss_naive(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -124.000000 analytic: -124.000000, relative error: 9.722173e-12
numerical: 344.400000 analytic: 344.400000, relative error: 1.263701e-11
numerical: 43.100000 analytic: 43.100000, relative error: 5.649351e-11
numerical: -19.000000 analytic: -19.000000, relative error: 9.117946e-12
numerical: 43.100000 analytic: 43.100000, relative error: 5.649351e-11
numerical: -75.300000 analytic: -75.300000, relative error: 3.981276e-11
numerical: 344.400000 analytic: 344.400000, relative error: 1.263701e-11
numerical: 12.500000 analytic: 12.500000, relative error: 2.155483e-10
numerical: 125.600000 analytic: 125.600000, relative error: 3.877414e-11
numerical: -55.600000 analytic: -55.600000, relative error: 9.225132e-11
numerical: -19.000000 analytic: -19.000000, relative error: 9.117946e-12
numerical: -13.900000 analytic: -13.900000, relative error: 2.711703e-10


In [24]:
#REGLARIZATION
loss, dW = svm_loss_naive(W, X, y, 2)

f = lambda W: svm_loss_naive(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -124.000238 analytic: -124.000238, relative error: 3.102774e-12
numerical: -111.700389 analytic: -111.700389, relative error: 7.398333e-12
numerical: 43.099809 analytic: 43.099809, relative error: 3.806605e-11
numerical: -13.900171 analytic: -13.900171, relative error: 2.696061e-10
numerical: 43.099809 analytic: 43.099809, relative error: 3.806605e-11
numerical: 143.000231 analytic: 143.000231, relative error: 6.471990e-12
numerical: 125.600265 analytic: 125.600265, relative error: 3.747817e-11
numerical: 12.500661 analytic: 12.500661, relative error: 2.246315e-10
numerical: 43.099809 analytic: 43.099809, relative error: 3.806605e-11
numerical: -269.100212 analytic: -269.100212, relative error: 3.927917e-12
numerical: 344.400288 analytic: 344.400288, relative error: 1.390104e-11
numerical: -124.000238 analytic: -124.000238, relative error: 3.102774e-12


In [100]:
def svm_loss_vectorized(W, X, y, alpha):
    """
    Multiclass SVM loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
    
    N, D = X.shape
    C = W.shape[1]
    
    S = X @ W
    
    idx = np.arange(N)
    
    mrg = np.maximum(0, S.T - S[idx,y] + 1).T
    mrg[idx,y] = 0
    
    mnn = np.where(mrg > 0, 1, 0)
    
    mtn = np.zeros_like(mnn)
    mtn[idx,y] = np.sum(mnn, axis=1)
    
    dW += X.T @ mnn
    dW -= X.T @ mtn
    
    loss = np.sum(mrg)
       
    loss += np.sum(alpha * W**2)
    dW += 2 * alpha * W

    
    return loss, dW

In [101]:
# NO REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 0.0)

f = lambda W: svm_loss_vectorized(W, X, y, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: -269.100000 analytic: -269.100000, relative error: 2.475678e-13
numerical: 143.000000 analytic: 143.000000, relative error: 9.641717e-12
numerical: 143.000000 analytic: 143.000000, relative error: 9.641717e-12
numerical: -75.300000 analytic: -75.300000, relative error: 2.067651e-12
numerical: 344.400000 analytic: 344.400000, relative error: 4.384404e-12
numerical: -269.100000 analytic: -269.100000, relative error: 2.475678e-13
numerical: -269.100000 analytic: -269.100000, relative error: 2.475678e-13
numerical: 43.100000 analytic: 43.100000, relative error: 9.450383e-12
numerical: 12.500000 analytic: 12.500000, relative error: 1.182578e-11
numerical: -13.900000 analytic: -13.900000, relative error: 6.668667e-11
numerical: 43.100000 analytic: 43.100000, relative error: 9.450383e-12
numerical: 143.000000 analytic: 143.000000, relative error: 9.641717e-12


In [102]:
# REGLARIZATION
loss, dW = svm_loss_vectorized(W, X, y, 2)

f = lambda W: svm_loss_vectorized(W, X, y, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-9)

numerical: 43.099809 analytic: 43.099809, relative error: 2.787813e-11
numerical: -18.999514 analytic: -18.999514, relative error: 3.103010e-11
numerical: -75.299889 analytic: -75.299889, relative error: 3.153003e-12
numerical: -111.700389 analytic: -111.700389, relative error: 7.397316e-12
numerical: -269.100212 analytic: -269.100212, relative error: 1.352961e-12
numerical: 143.000231 analytic: 143.000231, relative error: 6.471891e-12
numerical: 125.600265 analytic: 125.600265, relative error: 7.778390e-12
numerical: -18.999514 analytic: -18.999514, relative error: 3.103010e-11
numerical: -13.900171 analytic: -13.900171, relative error: 6.512491e-11
numerical: -13.900171 analytic: -13.900171, relative error: 6.512491e-11
numerical: -111.700389 analytic: -111.700389, relative error: 7.397316e-12
numerical: -55.599017 analytic: -55.599017, relative error: 2.220370e-11


## Gradient descent

In [103]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            X =  np.hstack( (np.ones((X.shape[0],1)), X) )

            
        N, d = X.shape
        
        C = (np.max(y) + 1) 
        if self.W is None: # Initialization
            self.W = 0.001 * np.random.randn(d, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            
            index = np.random.choice(N, batch_size, replace=False)
            X_batch = X[index, :]
            y_batch = y[index]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            self.W -= learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class LinearSVM(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return svm_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        if self.fit_intercept:
            X =  np.hstack( (np.ones((X.shape[0],1)), X) )
            
        y_pred = np.argmax(X @ self.W, axis=1)
        return y_pred

In [104]:
model = LinearSVM(fit_intercept=True)
model.train(X, y, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X)
model_accuracy = accuracy_score(y, pred)
print(model_accuracy)
assert model_accuracy > 0.97

iteration 0 / 75000: loss 128.253884
iteration 10000 / 75000: loss 5.002198
iteration 20000 / 75000: loss 2.164331
iteration 30000 / 75000: loss 0.502188
iteration 40000 / 75000: loss 4.648260
iteration 50000 / 75000: loss 1.211348
iteration 60000 / 75000: loss 3.889161
iteration 70000 / 75000: loss 3.227497
0.98
